In [35]:
## Importing Necessary Modules 

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [36]:
## getting the appropriate website parsed into hmtl
### Defining a function which takes in a url and outputs soup
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return(soup)


In [37]:
def extract_data(soup):
    headers = []
    data = []
    rows = soup.find_all('tr')
    header_row = rows[0]
    for header in header_row.find_all('th'):
        headers.append(header) 
        data.append([])
    for data_row in rows[1:]:
        row = data_row.find_all('td')
        for i in range(len(row)):
            if len(row) != len(data):
                continue
            else:
                data[i].append(row[i].text.strip())
    data_dict = {}
    for i in range(len(headers)):
        data_dict[headers[i]] = data[i]
    df = pd.DataFrame(data_dict)
    return(df)


In [40]:
all_stages_data = {}
stage_number = 1
url_basic = 'https://www.procyclingstats.com/race/tour-de-france/2023/stage-'
for stage_number in range(1,22):
    print(stage_number)
    url = url_basic + str(stage_number)
    soup = get_soup(url)
    all_stages_data[stage_number] = extract_data(soup)
print(all_stages_data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
{1:     [Rnk] [GC] [Timelag] [BIB] [H2H] [Specialty]  \
0       1    1     +0:00    19           Climber   
1       2    2     +0:08   161           Climber   
2       3    3     +0:18    11           Climber   
3       4    4     +0:22    37           Climber   
4       5    5     +0:22   151           Climber   
..    ...  ...       ...   ...   ...         ...   
171   172  172    +21:32   192            Sprint   
172   173  173    +21:32    64            Sprint   
173   174  174    +21:32   194           Classic   
174   175  175    +21:32   186            Sprint   
175   DNF                  131           Climber   

                                    [Rider] [Age]                 [Team]  \
0              YATES Adam UAE Team Emirates    30      UAE Team Emirates   
1              YATES Simon Team Jayco AlUla    30       Team Jayco AlUla   
2           POGAČAR Tadej UAE Team Emirates    24      UAE Team Emirates   
3            

In [42]:
df = pd.DataFrame(all_stages_data)

ValueError: If using all scalar values, you must pass an index